<p><br>
<center>
<b><h1>ALY6140 : Analytics System Technology</h1></b>
<b><h2>Northeastern University</h2></b>
<b><h2>Professor Daya Rudhramoorthi </h2></b>
<b><h2>Group 1 - Bhavik Bhosale, Sneha Rokkam</h2></b>
<b><h2>Capstone Group Project</h2></b>
<b><h2>Boston Property Assesment</h2></b>
</center>
</p>

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.options.display.float_format = '{:.2f}'.format


import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

## Read the Dataset

In [2]:
from group1_capstone import BostonPropertyDataFetcher
# Commenting the below code as the API fetches data in a loop of count 362. Saved the resultant dataframe to a csv and will read the data from the csv

url = 'https://data.boston.gov/datastore/odata3.0/1000d81c-5bb5-49e8-a9ab-44cd042f1db2?$format=xml'

# boston_data = BostonPropertyDataFetcher(url)

# assessment_data = boston_data.fetch_data()

# Read the CSV file
assessment_data = pd.read_csv('BostonProp.csv')

# Update column names
assessment_data.columns = assessment_data.columns.str[2:]

assessment_data.head(5)


,_id,PID,CM_ID,GIS_ID,ST_NUM,ST_NAME,UNIT_NUM,CITY,ZIP_CODE,BLDG_SEQ,NUM_BLDGS,LUC,LU,LU_DESC,BLDG_TYPE,OWN_OCC,OWNER,OWNERMAILADDRESS,RES_FLOOR,CD_FLOOR,RES_UNITS,COM_UNITS,RC_UNITS,LAND_SF,GROSS_AREA,LIVING_AREA,LAND_VALUE,BLDG_VALUE,TOTAL_VALUE,GROSS_TAX,YR_BUILT,YR_REMODEL,STRUCTURE_CLASS,ROOF_STRUCTURE,ROOF_COVER,INT_WALL,EXT_FNISHED,INT_COND,EXT_COND,OVERALL_COND,BED_RMS,FULL_BTH,HLF_BTH,KITCHENS,TT_RMS,BDRM_COND,BTHRM_STYLE1,BTHRM_STYLE2,BTHRM_STYLE3,KITCHEN_TYPE,KITCHEN_STYLE1,KITCHEN_STYLE2,KITCHEN_STYLE3,HEAT_TYPE,HEAT_FUEL,AC_TYPE,FIREPLACES,ORIENTATION,NUM_PARKING,PROP_VIEW,CORNER_UNIT
0,1,100001000,NaN,100001000,104.00,PUTNAM ST,NaN,EAST BOSTON,2128.00,1.00,1,105,R3,THREE-FAM DWELLING,RE - Row End,Y,PASCUCCI CARLO,"195 LEXINGTON ST, EAST BOSTON, MA 02128",3.00,NaN,NaN,NaN,NaN,1150.00,3353.00,2202.00,195700,588500,784200,8422.31,1900.00,NaN,NaN,F - Flat,C - Composition,N - Normal,A - Asbestos,A - Average,F - Fair,A - Average,6.00,3.00,0.00,3.00,12.00,NaN,S - Semi-Modern,S - Semi-Modern,S - Semi-Modern,3F - 3 Full Eat In Kitchens,S - Semi-Modern,S - Semi-Modern,S - Semi-Modern,W - Ht Water/Steam,NaN,N - None,0.00,NaN,3.00,A - Average,NaN
1,2,100002000,NaN,100002000,197.00,Lexington ST,NaN,EAST BOSTON,2128.00,1.00,1,105,R3,THREE-FAM DWELLING,RM - Row Middle,Y,SEMBRANO RODERICK,"197 LEXINGTON ST, EAST BOSTON, MA 02128",3.00,NaN,NaN,NaN,NaN,1150.00,3299.00,2307.00,196500,615300,811800,8718.73,1920.00,2000.00,NaN,F - Flat,C - Composition,N - Normal,M - Vinyl,A - Average,A - Average,A - Average,3.00,3.00,0.00,3.00,9.00,NaN,M - Modern,M - Modern,M - Modern,3F - 3 Full Eat In Kitchens,M - Modern,M - Modern,M - Modern,F - Forced Hot Air,NaN,C - Central AC,0.00,NaN,0.00,A - Average,NaN
2,3,100003000,NaN,100003000,199.00,Lexington ST,NaN,EAST BOSTON,2128.00,1.00,1,105,R3,THREE-FAM DWELLING,RM - Row Middle,N,GUERRA CHEVARRIA ANA S,"199 LEXINGTON ST, EAST BOSTON, MA 02128",3.00,NaN,NaN,NaN,NaN,1150.00,3392.00,2268.00,197100,599200,796300,8552.26,1905.00,1985.00,NaN,F - Flat,C - Composition,N - Normal,M - Vinyl,A - Average,G - Good,A - Average,5.00,3.00,0.00,3.00,13.00,NaN,M - Modern,M - Modern,M - Modern,3F - 3 Full Eat In Kitchens,S - Semi-Modern,S - Semi-Modern,S - Semi-Modern,S - Space Heat,NaN,N - None,0.00,NaN,0.00,A - Average,NaN
3,4,100004000,NaN,100004000,201.00,Lexington ST,NaN,EAST BOSTON,2128.00,1.00,1,105,R3,THREE-FAM DWELLING,RM - Row Middle,N,"MADDALENI JAMES E, TS","PO BOX 557 #, EVERETT, MA 02149",3.00,NaN,NaN,NaN,NaN,1150.00,3108.00,2028.00,197700,530200,727900,7817.65,1900.00,1991.00,NaN,M - Mansard,C - Composition,N - Normal,M - Vinyl,A - Average,A - Average,A - Average,5.00,3.00,0.00,3.00,11.00,NaN,S - Semi-Modern,S - Semi-Modern,S - Semi-Modern,3F - 3 Full Eat In Kitchens,S - Semi-Modern,S - Semi-Modern,S - Semi-Modern,W - Ht Water/Steam,NaN,N - None,0.00,NaN,0.00,A - Average,NaN
4,5,100005000,NaN,100005000,203.00,Lexington ST,NaN,EAST BOSTON,2128.00,1.00,1,104,R2,TWO-FAM DWELLING,RE - Row End,Y,MARKS TRAVIS JOSEPH,"203 Lexington ST, EAST BOSTON, MA 02128",3.00,NaN,NaN,NaN,NaN,2010.00,3700.00,2546.00,225700,569100,794800,8536.15,1900.00,1978.00,NaN,M - Mansard,C - Composition,N - Normal,M - Vinyl,A - Average,A - Average,A - Average,6.00,3.00,0.00,2.00,13.00,NaN,S - Semi-Modern,S - Semi-Modern,S - Semi-Modern,2F - 2 Full Eat In Kitchens,S - Semi-Modern,S - Semi-Modern,NaN,W - Ht Water/Steam,NaN,N - None,0.00,NaN,0.00,A - Average,NaN


In [3]:
assessment_data.shape

(180627, 61)

In [4]:
assessment_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180627 entries, 0 to 180626
Data columns (total 61 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   _id               180627 non-null  int64  
 1   PID               180627 non-null  int64  
 2   CM_ID             91160 non-null   float64
 3   GIS_ID            180627 non-null  int64  
 4   ST_NUM            171096 non-null  float64
 5   ST_NAME           180627 non-null  object 
 6   UNIT_NUM          80672 non-null   object 
 7   CITY              180622 non-null  object 
 8   ZIP_CODE          180624 non-null  float64
 9   BLDG_SEQ          180626 non-null  float64
 10  NUM_BLDGS         180627 non-null  int64  
 11  LUC               180627 non-null  int64  
 12  LU                180627 non-null  object 
 13  LU_DESC           180627 non-null  object 
 14  BLDG_TYPE         178101 non-null  object 
 15  OWN_OCC           180627 non-null  object 
 16  OWNER             18

## Data Preprocessing

In [9]:
from group1_capstone import DataProcessor

processor = DataProcessor()

processed_data = processor.process_data(assessment_data)


The duplicates found in the PID column: 
 PID                 0
CM_ID               0
GIS_ID              0
ST_NUM              0
ST_NAME             0
UNIT_NUM            0
CITY                0
ZIP_CODE            0
BLDG_SEQ            0
NUM_BLDGS           0
LUC                 0
LU                  0
LU_DESC             0
BLDG_TYPE           0
OWN_OCC             0
OWNER               0
OWNERMAILADDRESS    0
RES_FLOOR           0
CD_FLOOR            0
RES_UNITS           0
COM_UNITS           0
RC_UNITS            0
LAND_SF             0
GROSS_AREA          0
LIVING_AREA         0
LAND_VALUE          0
BLDG_VALUE          0
TOTAL_VALUE         0
GROSS_TAX           0
YR_BUILT            0
YR_REMODEL          0
STRUCTURE_CLASS     0
ROOF_STRUCTURE      0
ROOF_COVER          0
INT_WALL            0
EXT_FNISHED         0
INT_COND            0
EXT_COND            0
OVERALL_COND        0
BED_RMS             0
FULL_BTH            0
HLF_BTH             0
KITCHENS            0
TT_RMS      

KeyError: "['_id'] not found in axis"

In [10]:
processed_data.head(5)

,PID,CM_ID,GIS_ID,ST_NUM,ST_NAME,UNIT_NUM,CITY,ZIP_CODE,BLDG_SEQ,NUM_BLDGS,LUC,LU,LU_DESC,BLDG_TYPE,OWN_OCC,OWNER,OWNERMAILADDRESS,RES_FLOOR,CD_FLOOR,RES_UNITS,COM_UNITS,RC_UNITS,LAND_SF,GROSS_AREA,LIVING_AREA,LAND_VALUE,BLDG_VALUE,TOTAL_VALUE,GROSS_TAX,YR_BUILT,YR_REMODEL,STRUCTURE_CLASS,ROOF_STRUCTURE,ROOF_COVER,INT_WALL,EXT_FNISHED,INT_COND,EXT_COND,OVERALL_COND,BED_RMS,FULL_BTH,HLF_BTH,KITCHENS,TT_RMS,BDRM_COND,BTHRM_STYLE1,BTHRM_STYLE2,BTHRM_STYLE3,KITCHEN_TYPE,KITCHEN_STYLE1,KITCHEN_STYLE2,KITCHEN_STYLE3,HEAT_TYPE,HEAT_FUEL,AC_TYPE,FIREPLACES,ORIENTATION,NUM_PARKING,PROP_VIEW,CORNER_UNIT
0,100001000,NaN,100001000,104.00,PUTNAM ST,NaN,EAST BOSTON,2128.00,1.00,1,105,R3,THREE-FAM DWELLING,RE - Row End,Y,PASCUCCI CARLO,"195 LEXINGTON ST, EAST BOSTON, MA 02128",3.00,NaN,NaN,NaN,NaN,1150.00,3353.00,2202.00,195700,588500,784200,8422.31,1900.00,NaN,NaN,F - Flat,C - Composition,N - Normal,A - Asbestos,A - Average,F - Fair,A - Average,6.00,3.00,0.00,3.00,12.00,NaN,S - Semi-Modern,S - Semi-Modern,S - Semi-Modern,3F - 3 Full Eat In Kitchens,S - Semi-Modern,S - Semi-Modern,S - Semi-Modern,W - Ht Water/Steam,NaN,N - None,0.00,NaN,3.00,A - Average,NaN
1,100002000,NaN,100002000,197.00,Lexington ST,NaN,EAST BOSTON,2128.00,1.00,1,105,R3,THREE-FAM DWELLING,RM - Row Middle,Y,SEMBRANO RODERICK,"197 LEXINGTON ST, EAST BOSTON, MA 02128",3.00,NaN,NaN,NaN,NaN,1150.00,3299.00,2307.00,196500,615300,811800,8718.73,1920.00,2000.00,NaN,F - Flat,C - Composition,N - Normal,M - Vinyl,A - Average,A - Average,A - Average,3.00,3.00,0.00,3.00,9.00,NaN,M - Modern,M - Modern,M - Modern,3F - 3 Full Eat In Kitchens,M - Modern,M - Modern,M - Modern,F - Forced Hot Air,NaN,C - Central AC,0.00,NaN,0.00,A - Average,NaN
2,100003000,NaN,100003000,199.00,Lexington ST,NaN,EAST BOSTON,2128.00,1.00,1,105,R3,THREE-FAM DWELLING,RM - Row Middle,N,GUERRA CHEVARRIA ANA S,"199 LEXINGTON ST, EAST BOSTON, MA 02128",3.00,NaN,NaN,NaN,NaN,1150.00,3392.00,2268.00,197100,599200,796300,8552.26,1905.00,1985.00,NaN,F - Flat,C - Composition,N - Normal,M - Vinyl,A - Average,G - Good,A - Average,5.00,3.00,0.00,3.00,13.00,NaN,M - Modern,M - Modern,M - Modern,3F - 3 Full Eat In Kitchens,S - Semi-Modern,S - Semi-Modern,S - Semi-Modern,S - Space Heat,NaN,N - None,0.00,NaN,0.00,A - Average,NaN
3,100004000,NaN,100004000,201.00,Lexington ST,NaN,EAST BOSTON,2128.00,1.00,1,105,R3,THREE-FAM DWELLING,RM - Row Middle,N,"MADDALENI JAMES E, TS","PO BOX 557 #, EVERETT, MA 02149",3.00,NaN,NaN,NaN,NaN,1150.00,3108.00,2028.00,197700,530200,727900,7817.65,1900.00,1991.00,NaN,M - Mansard,C - Composition,N - Normal,M - Vinyl,A - Average,A - Average,A - Average,5.00,3.00,0.00,3.00,11.00,NaN,S - Semi-Modern,S - Semi-Modern,S - Semi-Modern,3F - 3 Full Eat In Kitchens,S - Semi-Modern,S - Semi-Modern,S - Semi-Modern,W - Ht Water/Steam,NaN,N - None,0.00,NaN,0.00,A - Average,NaN
4,100005000,NaN,100005000,203.00,Lexington ST,NaN,EAST BOSTON,2128.00,1.00,1,104,R2,TWO-FAM DWELLING,RE - Row End,Y,MARKS TRAVIS JOSEPH,"203 Lexington ST, EAST BOSTON, MA 02128",3.00,NaN,NaN,NaN,NaN,2010.00,3700.00,2546.00,225700,569100,794800,8536.15,1900.00,1978.00,NaN,M - Mansard,C - Composition,N - Normal,M - Vinyl,A - Average,A - Average,A - Average,6.00,3.00,0.00,2.00,13.00,NaN,S - Semi-Modern,S - Semi-Modern,S - Semi-Modern,2F - 2 Full Eat In Kitchens,S - Semi-Modern,S - Semi-Modern,NaN,W - Ht Water/Steam,NaN,N - None,0.00,NaN,0.00,A - Average,NaN


In [11]:
processed_data.shape

(180448, 60)

### Descriptive Statistics

In [12]:
desc_result = processor.describe_numerical_columns(processed_data)
desc_result

,PID,CM_ID,GIS_ID,ST_NUM,ZIP_CODE,BLDG_SEQ,NUM_BLDGS,LUC,RES_FLOOR,CD_FLOOR,RES_UNITS,COM_UNITS,RC_UNITS,LAND_SF,GROSS_AREA,LIVING_AREA,LAND_VALUE,BLDG_VALUE,TOTAL_VALUE,GROSS_TAX,YR_BUILT,YR_REMODEL,BED_RMS,FULL_BTH,HLF_BTH,KITCHENS,TT_RMS,FIREPLACES,NUM_PARKING
count,180448.00,91158.00,180448.00,170920.00,180445.00,180447.00,180448.00,180448.00,146944.00,70555.00,10329.00,623.00,37.00,172903.00,145816.00,145526.00,180448.00,180448.00,180448.00,180448.00,157518.00,85330.00,132164.00,169333.00,169486.00,169335.00,132098.00,169489.00,132250.00
mean,1141872743.02,916700965.96,1141873828.03,226.84,2129.84,1.00,1.02,202.66,1.88,3.51,6.84,3.03,3.24,7810.99,5386.36,4369.60,376652.42,1120716.28,1500358.13,16932.77,1932.34,2001.32,3.15,1.35,0.22,1.05,6.96,0.27,1.34
std,708114503.99,687936774.77,708115414.47,477.35,30.77,0.03,0.46,265.92,1.12,5.27,18.38,10.53,6.13,263198.82,41402.56,38494.85,4424357.56,10808892.37,14123131.29,220704.56,63.72,65.12,2.10,1.05,0.46,0.81,4.02,0.62,2.42
min,100001000.00,100018000.00,100001000.00,0.00,2026.00,1.00,1.00,13.00,0.00,0.00,1.00,1.00,1.00,100.00,3.00,2.00,0.00,0.00,0.00,0.00,1700.00,201.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00
25%,501666750.00,306925000.00,501667000.00,24.00,2119.00,1.00,1.00,102.00,1.00,1.00,2.00,1.00,1.00,999.00,973.00,947.00,0.00,298500.00,379400.00,3879.29,1900.00,1987.00,2.00,1.00,0.00,1.00,4.00,0.00,0.00
50%,1102683001.00,602642400.00,1102683000.00,68.00,2127.00,1.00,1.00,102.00,2.00,2.00,3.00,1.00,1.00,2008.00,2122.00,1492.00,0.00,488900.00,609227.00,6456.59,1920.00,2004.00,3.00,1.00,0.00,1.00,6.00,0.00,1.00
75%,1810485250.00,1602267000.00,1810485250.00,213.00,2131.00,1.00,1.00,108.00,2.50,4.00,5.00,2.00,2.00,4764.00,4035.00,2611.20,224700.00,744300.00,913100.00,9703.55,1964.00,2015.00,4.00,2.00,0.00,1.00,9.00,0.00,2.00
max,2205670000.00,2205665000.00,2205670000.00,5341.00,2467.00,4.00,119.00,995.00,62.00,60.00,477.00,212.00,29.00,101513565.00,6982322.00,6982322.00,486046900.00,1723255900.00,2143052600.00,23935695.62,20198.00,20220.00,17.00,17.00,7.00,5.00,20.00,12.00,210.00


In [ ]:
# Count total missing values at each column in DataFrame
missing_values_count = assesment_data.isnull().sum()

# Calculate percentage of missing values at each column in DataFrame
missing_values_percentage = (assesment_data.isnull().sum() / len(assesment_data)) * 100

# Create a new DataFrame with the column name, missing value count and missing value percentage
missing_data_df = pd.DataFrame({
    'Missing Value Count': missing_values_count,
    'Missing Value Percentage': missing_values_percentage
})

# Format the 'Missing Value Percentage' column as a percentage with 2 decimal places
missing_data_df['Missing Value Percentage'] = missing_data_df['Missing Value Percentage'].apply(lambda x: f'{x:.2f}%')

print(missing_data_df)

### Drop Unwanted Columns

In [ ]:
#List out unwanted columns
unwanted_columns = ['PID',
                    'CM_ID',
                    'GIS_ID',
                    'ST_NUM',
                    'ST_NAME',
                    'UNIT_NUM',
                    'LUC',
                    'LU',
                    'LU_DESC',
                    'OWNER MAIL ADDRESS',
                    'RES_FLOOR',
                    'RES_UNITS',
                    'COM_UNITS',
                    'LAND_SF',
                    'LAND_VALUE',
                    'YR_REMODEL',
                    'STRUCTURE_CLASS',
                    'ROOF_STRUCTURE',
                    'ROOF_COVER',
                    'RC_UNITS',
                    'INT_WALL',
                    'INT_COND',
                    'BED_RMS',
                    'FULL_BTH',
                    'HLF_BTH',
                    'BDRM_COND',
                    'BTHRM_STYLE1',
                    'BTHRM_STYLE2',
                    'BTHRM_STYLE3',
                    'KITCHEN_STYLE1',
                    'KITCHEN_STYLE2',
                    'KITCHEN_STYLE3',
                    'HEAT_FUEL',
                    'FIREPLACES',
                    'NUM_PARKING',
                    'PROP_VIEW',
                    'CORNER_UNIT',
                    'OVERALL_COND',
                    'ORIENTATION',
                    'CD_FLOOR'
                   ]

# Drop these rows from dataFrame
property_data = assesment_data.drop(unwanted_columns, inplace = False, axis='columns')

In [ ]:
property_data.head(5)

In [ ]:
# Check the percentage and count of the missing data 
for col in property_data.columns:
    missing_count = sum(property_data[col].isnull())
    missing_percentage = round(np.mean(property_data[col].isnull()) * 100, 2)
    print('{:<15s}\t: {:<5d}\t: {:<5.2f}%'.format(col, missing_count, missing_percentage))


In [ ]:
property_data.dropna(subset=['AC_TYPE'],inplace=True)

In [ ]:
# Check the percentage and count of the missing data 

def print_missing_value_summary(df):
    for col in df.columns:
        pct_missing = np.mean(df[col].isnull())
        print('{:<15s} : {:<5d} \t: {:.2f}%'.format(col, sum(df[col].isnull()), round(pct_missing * 100, 2)))

print_missing_value_summary(property_data)


### Dropping Rows with Nulls

In [ ]:
# Droping the rows with null values.

property_data.dropna(inplace=True)


In [ ]:
# Check the percentage and count of the missing data 

def print_missing_value_summary(df):
    for col in df.columns:
        pct_missing = np.mean(df[col].isnull())
        print('{:<15s} : {:<5d} \t: {:.2f}%'.format(col, sum(df[col].isnull()), round(pct_missing * 100, 2)))

print_missing_value_summary(property_data)

In [ ]:
property_data.shape

### Categorical Variables 

In [ ]:
columns = ['CITY', 'NUM_BLDGS', 'BLDG_TYPE', 'YR_BUILT', 'EXT_COND', 'TT_RMS',
           'KITCHEN_TYPE', 'HEAT_TYPE', 'AC_TYPE']

for col in columns:
    print("# # # # #  {}  # # # # #".format(col))
    display(property_data[col].value_counts())


### Descriptive Statistics

In [ ]:
numerical_columns = property_data.select_dtypes(include=['int64', 'float64']).columns
property_data[numerical_columns].describe(datetime_is_numeric=True,include="all")

### EDA 

In [ ]:
#Distribution of Year Built
sns.set(style="darkgrid")
plt.figure(figsize=(10, 6))
sns.histplot(data=property_data['YR_BUILT'], kde=True)
plt.xlabel('Year Built')
plt.ylabel('Count')
plt.title('Distribution of Year Built')
plt.xlim(1825, 2023)  # Set x-axis limits 
plt.show()

In [ ]:
#Boxplot to check outliers of Year Built
sns.set(style="darkgrid")
plt.figure(figsize=(10, 6))
sns.boxplot(data=property_data['YR_BUILT'], color='skyblue')
plt.xlabel('Year Built')
plt.ylabel('Year')
plt.title('Boxplot of Year Built')
plt.ylim(1700, 2200) 
plt.show()

In [ ]:
# sns.set(style="whitegrid")
# plt.figure(figsize=(10, 6))
# sns.boxplot(data=property_data['TOTAL_VALUE'], color='skyblue', showfliers=False)
# plt.xlabel('Total Value')
# plt.ylabel('Value')
# plt.title('Boxplot of Total Value')
# plt.xticks(rotation=45) 
# plt.show()

sns.boxplot(data=property_data['TOTAL_VALUE'])

### Handle Outliers 

In [ ]:
#Display outliers in TOTAL_VALUE
filtered_data = property_data.loc[property_data['TOTAL_VALUE'] > 2100000, 'TOTAL_VALUE']
filtered_data

In [ ]:
#Drop the outliers 
property_data = property_data[property_data['TOTAL_VALUE'] <= 2000000]
value_counts = property_data['TOTAL_VALUE'].value_counts()

In [ ]:
#Distribution of TOTAL_VALUE
sns.set(style="darkgrid")
plt.figure(figsize=(10, 6))
sns.histplot(data=property_data['TOTAL_VALUE'], kde=True, color='skyblue', bins=20)
plt.xlabel('Total Value')
plt.ylabel('Count')
plt.title('Distribution of Total Value')
plt.xticks(rotation=45)
plt.show()

In [ ]:
sns.set(style="whitegrid")
plt.figure(figsize=(8, 6))
sns.boxplot(data=property_data['GROSS_AREA'], color='skyblue', showfliers=False)
plt.xlabel('Gross Area')
plt.ylabel('Area')
plt.title('Boxplot of Gross Area')
plt.xticks(rotation=45)
plt.ylim(0, 50000) 
plt.show()

In [ ]:
## Removing outliars in GROSS_AREA that is values above 25000
filtered_data = property_data[property_data['GROSS_AREA'] < 25000]['GROSS_AREA']

sns.set(style="whitegrid")
plt.figure(figsize=(8, 6))
sns.boxplot(data=filtered_data, color='skyblue', showfliers=False)
plt.xlabel('Gross Area')
plt.ylabel('Area')
plt.title('Boxplot of Gross Area (Filtered)')
plt.xticks(rotation=45)
plt.ylim(0, 25000) 
plt.show()

In [ ]:
sns.set(style="whitegrid")
plt.figure(figsize=(10, 6))
sns.histplot(data=property_data['GROSS_AREA'], kde=True, color='skyblue', bins=30)
plt.xlabel('Gross Area')
plt.ylabel('Count')
plt.title('Distribution of Gross Area')
plt.xticks(rotation=45)
plt.show()

In [ ]:
sns.set(style="whitegrid")
plt.figure(figsize=(10, 6))
sns.histplot(data=property_data['GROSS_TAX'], kde=True, color='skyblue', bins=20)
plt.xlabel('Gross Tax')
plt.ylabel('Count')
plt.title('Distribution of Gross Tax')
plt.xticks(rotation=45)
plt.show()